#prerun

In [ ]:
!pip install tslearn
!pip install mplfinance
!pip install dtw

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import math
from io import StringIO
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import os
from joblib import load
import csv
import shutil
from sklearn.neighbors import NearestNeighbors
import glob

##for colab only

In [ ]:
from google.colab import files
from google.colab import drive

#main

In [ ]:


#functions

#preprocess data to points
def pips(data: np.array, n_pips: int, dist_measure: int):

    # dist_measure
    # 1 = Euclidean Distance
    # 2 = Perpindicular Distance
    # 3 = Vertical Distance




    #first phase converting
    n_pips = n_pips + 2


    pips_x = [0, len(data) - 1]  # Index
    pips_y = [data[0], data[-1]] # Price

    for curr_point in range(2, n_pips):

        md = 0.0 # Max distance
        md_i = -1 # Max distance index
        insert_index = -1

        for k in range(0, curr_point - 1):

            # Left adjacent, right adjacent indices
            left_adj = k
            right_adj = k + 1

            time_diff = pips_x[right_adj] - pips_x[left_adj]
            price_diff = pips_y[right_adj] - pips_y[left_adj]
            slope = price_diff / time_diff
            intercept = pips_y[left_adj] - pips_x[left_adj] * slope;

            for i in range(pips_x[left_adj] + 1, pips_x[right_adj]):

                d = 0.0 # Distance
                if dist_measure == 1: # Euclidean distance
                    d =  ( (pips_x[left_adj] - i) ** 2 + (pips_y[left_adj] - data[i]) ** 2 ) ** 0.5
                    d += ( (pips_x[right_adj] - i) ** 2 + (pips_y[right_adj] - data[i]) ** 2 ) ** 0.5
                elif dist_measure == 2: # Perpindicular distance
                    d = abs( (slope * i + intercept) - data[i] ) / (slope ** 2 + 1) ** 0.5
                else: # Vertical distance
                    d = abs( (slope * i + intercept) - data[i] )

                if d > md:
                    md = d
                    md_i = i
                    insert_index = right_adj

        pips_x.insert(insert_index, md_i)
        pips_y.insert(insert_index, data[md_i])



     #second phase converting
    points_no = len(pips_x)  #number of points
    points = [] #list of new converting data


    #the rate of change of y over the rate of change of x for every point
    for i in range(2,points_no) :

      x = ((pips_x[i] - pips_x[i-1])/ pips_x[i-1] ) * 100
      y = ((pips_y[i] - pips_y[i-1])/ pips_y[i-1] ) * 100

      z = y / x

      points.append(z)





    return points



#creating points dataset
def create_dataset(csv_file,no_of_points_in_raw,window_size=24,step=5):

    #load data
    data = pd.read_csv(csv_file)
    data['date'] = data['date'].astype('datetime64[s]')
    data = data.set_index('date')


    #dataset
    x = data['close'].to_numpy()



    #create dataset
    dataset = pd.DataFrame()


    # Iterate through the data
    for i in range(0, len(x) - window_size + 1, step):
        window_data = x[i:i+window_size]


        #get points
        points = pips(window_data, no_of_points_in_raw, 2)


        #creating the new record
        new_record = {
            'points': points

        }

        # Convert the new record to a DataFrame
        new_df = pd.DataFrame([new_record])

        # Concatenate the existing DataFrame with the new record DataFrame
        dataset = pd.concat([dataset, new_df], ignore_index=True)

    # Save the updated DataFrame to CSV
    dataset.to_csv('dataset.csv', index=False)

    #return
    return dataset


#clustering and saving clusters data
def cluster_and_save(dataset, n_clusters):
    # Preprocess data (scaling)
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(dataset)

    # Apply K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(scaled_data)

    # Create a folder to save clusters if it doesn't exist
    if not os.path.exists('clusters'):
        os.makedirs('clusters')

    # Convert numpy array to DataFrame
    dataset_df = pd.DataFrame(dataset, columns=[f'Feature_{i+1}' for i in range(dataset.shape[1])])

    # Iterate over each cluster
    for cluster_num in range(n_clusters):
        # Filter data points belonging to the current cluster
        cluster_data = dataset_df[cluster_labels == cluster_num]

        # Convert each row to a list and save as a single row containing a list of features
        cluster_data_list = cluster_data.values.tolist()

        # Save the cluster data to a file
        file_name = f'cluster_{cluster_num + 1}.csv'
        file_path = os.path.join('clusters', file_name)
        with open(file_path, 'w') as file:
            for row in cluster_data_list:
                file.write(','.join(map(str, row)) + '\n')
        print(f"Cluster {cluster_num + 1} saved to: {file_path}")


#determine the best number of clusters
def determine_optimal_clusters(dataset, cluster_range):
    # Preprocess data (scaling)
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(dataset)

    # Initialize variables to store silhouette scores
    silhouette_scores = []

    # Iterate over different number of clusters
    for n_clusters in cluster_range:
        # Apply K-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(scaled_data)

        # Calculate silhouette score
        silhouette_avg = silhouette_score(scaled_data, cluster_labels)
        silhouette_scores.append(silhouette_avg)

    # Find the number of clusters with the highest silhouette score
    optimal_clusters = cluster_range[np.argmax(silhouette_scores)]

    return optimal_clusters




#full clustering sequence
def clustering(data_file,row_points,window,step,range_clusters=0,no_of_clusters=0):

  #preprocessing
  create_dataset(data_file,row_points,window,step)

  print("preprocessing done !")

  # Load dataset from CSV
  dataset = pd.read_csv('dataset.csv')

  # Convert string representation of points to lists of floats
  dataset['points'] = dataset['points'].apply(eval)

  # Convert points to numpy array
  points_array = np.array(dataset['points'].tolist())

  print("load points done !")


  if no_of_clusters == 0 :

    # Load dataset from CSV
    dataset = pd.read_csv('dataset.csv')

    # Convert string representation of points to lists of floats
    dataset['points'] = dataset['points'].apply(eval)



    # Convert points to numpy array
    points_array = np.array(dataset['points'].tolist())



    # Define range of cluster numbers to test
    cluster_range = range_clusters

    # Determine the optimal number of clusters using the Silhouette Method
    optimal_clusters = determine_optimal_clusters(points_array, cluster_range)

    print("Optimal number of clusters:", optimal_clusters)



    # Define the number of clusters
    n_clusters = int(optimal_clusters)


  elif no_of_clusters != 0 :
    print("start clustering to the chosen clusters number")
    n_clusters = no_of_clusters

  else:
    print("error please choose a range or a number of cluster to start clustering !")

  print("start clustering")

  # Perform clustering and save clusters
  cluster_and_save(points_array, n_clusters)

  print("clustering done !")


#to plot clusters and visualize patterns
def plot_clusters(cluster_folder='clusters',points=5):
    # Get list of cluster files
    cluster_files = [file for file in os.listdir(cluster_folder) if file.startswith('cluster_')]

    # Plot each cluster separately
    for file in cluster_files:
        # Read cluster data from CSV
        cluster_data = pd.read_csv(os.path.join(cluster_folder, file))

        # Plot cluster data
        plt.figure(figsize=(10, points+1))
        for i in range(len(cluster_data)):
            plt.plot(cluster_data.iloc[i].values, label=f'Record {i+1}')  # Plot each record separately

        plt.xlabel('Feature Index')
        plt.ylabel('Feature Value')
        plt.title(f'Cluster {file[:-4]} Records Overlay Plot')

        plt.grid(True)
        plt.show()


#to save plots
def save_cluster_plots(cluster_folder='clusters', points=5, save_folder='cluster_plots'):
    if not os.path.isdir(save_folder):
        os.makedirs(save_folder)

    # Get list of cluster files
    cluster_files = [file for file in os.listdir(cluster_folder) if file.startswith('cluster_')]

    # Plot each cluster separately
    for file in cluster_files:
        # Read cluster data from CSV
        cluster_data = pd.read_csv(os.path.join(cluster_folder, file))

        # Plot cluster data
        plt.figure(figsize=(10, points+1))
        for i in range(len(cluster_data)):
            plt.plot(cluster_data.iloc[i].values, label=f'Record {i+1}')  # Plot each record separately

        plt.xlabel('Feature Index')
        plt.ylabel('Feature Value')
        plt.title(f'Cluster {file[:-4]} Records Overlay Plot')

        plt.grid(True)

        plt.savefig(os.path.join(save_folder, f'cluster_{file[:-4]}.png'))  # Save plot as PNG file
        plt.close()




def load_clusters(folder_path, no_of_clusters):
    """Load cluster data from CSV files in the specified folder."""
    cluster_data = []
    # Get a list of files in the folder
    files = os.listdir(folder_path)
    # Iterate over the files
    for file_name in files:
        # Check if the file is a CSV file and matches the cluster naming convention
        if file_name.endswith('.csv') and file_name.startswith('cluster_'):
            # Extract the cluster number from the file name
            cluster_number = int(file_name.split('_')[1].split('.')[0])
            # Check if the cluster number is within the specified range
            if cluster_number <= no_of_clusters:
                file_path = os.path.join(folder_path, file_name)
                cluster_df = pd.read_csv(file_path)
                cluster_data.append(cluster_df)
    return cluster_data

#cluster detection
def detect_cluster(new_record, cluster_data, k):
    """Detect the cluster that a new record belongs to using KNN."""
    distances = []
    for cluster_df in cluster_data:
        num_samples = cluster_df.shape[0]  # Get the number of samples in the cluster dataset
        if k > num_samples:
            k = num_samples  # Adjust k if it's greater than the number of samples
        X = cluster_df.values  # Convert cluster data to numpy array
        nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(X)
        distances.append(nbrs.kneighbors([new_record])[0][0].mean())
    # Assign the new record to the cluster with the smallest average distance
    cluster_index = np.argmin(distances)
    return cluster_index + 1  # Return cluster index (starting from 1)




#count clusters
def count_cluster(folder_path="clusters"):
    if not os.path.isdir(folder_path):
        return "Invalid folder path"

    file_count = 0
    for file in os.listdir(folder_path):
        if file.startswith("cluster_") and file.endswith(".csv"):
            file_count += 1


    print(f'number of cluster = {file_count}')
    return file_count

#delete the empty clusters
def delete_empty(folder_path="clusters"):
    if not os.path.isdir(folder_path):
        return "Invalid folder path"

    files_to_rename = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            with open(file_path, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                num_records = sum(1 for _ in csv_reader)  # Count the number of records
                if num_records < 2:
                    os.remove(file_path)
                    print(f"Deleted {file} due to less than 2 records.")
                else:
                    files_to_rename.append(file)



def organize(folder_path="clusters"):

  data = os.path.abspath(folder_path)
  for i, f in enumerate(os.listdir(data)):
      src = os.path.join(data, f)
      dst = os.path.join(data, (f"cluster_{str(i + 1)}.csv"))
      os.rename(src, dst)





#full detection sequence
def detect(records,no_of_clusters,no_of_points_in_raw,clusters_folder="clusters" ):


  #loading clusters
  cluster_data = load_clusters(clusters_folder,no_of_clusters)


  #dataset
  x = np.array(records)


  #get points
  points = pips(x, no_of_points_in_raw, 2)


  #predict cluster
  new_record = np.array(points)
  predicted_cluster = detect_cluster(new_record, cluster_data,no_of_clusters)


  #printing
  print(f"cluster : {predicted_cluster}")

  return predicted_cluster



def clear_clusters(folder_path="clusters"):
    if not os.path.isdir(folder_path):
        print("Folder does not exist.")
        return

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)  # Delete file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete directory and its contents
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")






def download_plots(folder="cluster_plots"):
  files.download(f"{folder}")






#variables

In [ ]:
#first clustering
data = "sol.csv" #data file (solana)
points = 5 #number of points in each record
window = 24 #window size
step = 10 #step size between every window and the next
clusters = range(25,50) #range of clusters number

#clustering

In [ ]:

#clear old clusters
clear_clusters()

#clustering
clustering(data ,points ,window ,step ,clusters )

#cleaning empty clusters
delete_empty()


#get the number of clusters after cleaning
clusters = count_cluster()



preprocessing done !
load points done !


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

Optimal number of clusters: 43
start clustering


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 1 saved to: clusters/cluster_1.csv
Cluster 2 saved to: clusters/cluster_2.csv
Cluster 3 saved to: clusters/cluster_3.csv
Cluster 4 saved to: clusters/cluster_4.csv
Cluster 5 saved to: clusters/cluster_5.csv
Cluster 6 saved to: clusters/cluster_6.csv
Cluster 7 saved to: clusters/cluster_7.csv
Cluster 8 saved to: clusters/cluster_8.csv
Cluster 9 saved to: clusters/cluster_9.csv
Cluster 10 saved to: clusters/cluster_10.csv
Cluster 11 saved to: clusters/cluster_11.csv
Cluster 12 saved to: clusters/cluster_12.csv
Cluster 13 saved to: clusters/cluster_13.csv
Cluster 14 saved to: clusters/cluster_14.csv
Cluster 15 saved to: clusters/cluster_15.csv
Cluster 16 saved to: clusters/cluster_16.csv
Cluster 17 saved to: clusters/cluster_17.csv
Cluster 18 saved to: clusters/cluster_18.csv
Cluster 19 saved to: clusters/cluster_19.csv
Cluster 20 saved to: clusters/cluster_20.csv
Cluster 21 saved to: clusters/cluster_21.csv
Cluster 22 saved to: clusters/cluster_22.csv
Cluster 23 saved to: cluster

#ploting clusters

In [ ]:
# plotting if needed
plot_clusters()


In [ ]:
#clear old images
clear_clusters("cluster_plots")


#save plots
save_cluster_plots()

#cluster detection

In [ ]:
#get the number of clusters after cleaning
clusters = count_cluster()


#pattern detection 24 record
close_prices = [
    19.845, 19.861, 19.886, 19.823, 19.826, 19.806, 19.765, 19.78, 19.763, 19.765,
    19.826, 19.832, 19.881, 19.848, 19.77, 19.805, 19.779, 19.786, 19.663, 19.672,
    19.576, 19.539, 19.499, 19.374
]





the_detected_clustesr = detect(close_prices,clusters,points)



cluster : 25
